#### This class is designed for the Borusan Hackathon to efficiently extract and segment text from PDF documents.

In [8]:
!pip install pdfplumber
!pip install pdfplumber

In [9]:
import pdfplumber

class BorusanHackathonPDFExtractor:
    def __init__(self, file_path, chunk_size=500):
        """
        Initializes the BorusanHackathonPDFExtractor.

        Args:
        file_path (str): The path to the PDF file for the Borusan Hackathon.
        chunk_size (int): Maximum size of each text chunk.
        """
        self.file_path = file_path
        self.chunk_size = chunk_size
        self.full_text = ""
        self.chunks = []

    def extract_text(self):
        """Extracts text from the PDF file."""
        with pdfplumber.open(self.file_path) as pdf:
            for page in pdf.pages:
                self.full_text += page.extract_text() or ""

    def split_into_chunks(self):
        """
        Splits the extracted text into chunks of up to chunk_size.
        Chunks are split at the last period before the chunk_size limit.
        """
        text = self.full_text
        while len(text) > self.chunk_size:
            last_period_index = text[:self.chunk_size].rfind('.')
            if last_period_index == -1:
                last_period_index = self.chunk_size
            self.chunks.append(text[:last_period_index].strip())
            text = text[last_period_index+1:]

        if text:
            self.chunks.append(text.strip())

    def get_chunks(self):
        """Returns the list of text chunks."""
        return self.chunks

    def process_pdf(self):
        """Processes the PDF file to extract and split text."""
        self.extract_text()
        self.split_into_chunks()

### Get chunks

In [10]:
extractor = BorusanHackathonPDFExtractor("source/borusan.pdf")
extractor.process_pdf()
chunks = extractor.get_chunks()

## Download required libraries

In [ ]:
!pip install openai python-dotenv
!pip install --upgrade qdrant_client

#### see openai version below (for reference)

In [12]:
import openai 
print(openai.VERSION)

1.3.8


### Initialize Azure OpenAI client with API credentials loaded from environment variables.

In [13]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

load_dotenv() 

API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
RESOURCE_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')

client = AzureOpenAI(
  api_key=API_KEY,
  api_version="2023-05-15",
  azure_endpoint=RESOURCE_ENDPOINT
)

##### Configuration for the embedding model: model name - openai embedding model

In [23]:
embedding_model = "text-embedding-ada-002" 

##### Convert text chunks to PointStructs with embeddings for Qdrant indexing.

In [15]:
from qdrant_client.http.models import PointStruct

def create_point_structs_from_chunks(chunks):
    """
    Verilen metin parçalarından (chunks) PointStruct nesneleri oluşturur.
    
    Args:
    chunks (list of str): Metin parçalarının listesi.

    Returns:
    list of PointStruct: Oluşturulan PointStruct nesnelerinin listesi.
    """
    points = []

    for index, chunk in enumerate(chunks, start=1):
        embeddings = client.embeddings.create(input=chunk, model=embedding_model).data[0].embedding
        points.append(PointStruct(id=index, vector=embeddings, payload={"text": chunk}))

    print("Embedding size:", len(embeddings)) # VectorParams fonksiyonu için embedding size'ı döndürmemiz gerekiyor. Buna göre size parametresini ayarlayabilirsiniz.

    return points

#### Get points

In [16]:
points = create_point_structs_from_chunks(chunks)  

Embedding size: 1536


#####  Create a client instance for Qdrant

In [17]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333) 

#### Create collection with vector configuration - Cosine distance and 1536 vector size

In [18]:
from qdrant_client.http.models import VectorParams, Distance

client.recreate_collection(
    collection_name="AutoHackathon",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

True

####  Upsert points to the collection

In [20]:
client.upsert(
    collection_name="AutoHackathon",
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

#### see collection info or open http://localhost:6333/collections/AutoHackathon in browser for more details

In [25]:
collection_info = client.get_collection(collection_name="AutoHackathon")
print("Number of vectors in the Coderspace - Auto Hackathon collection:", collection_info.vectors_count)

Number of vectors in the Coderspace - Auto Hackathon collection: 66
